In [11]:
%pip install agentpy
%pip install owlready2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [12]:
from owlready2 import *
# Model design
import agentpy as ap
import numpy as np
# Visualization
import seaborn as sns

In [13]:
#Tienda

onto = get_ontology("wealth_ontology.owl").load()

#if onto is not None:
#    onto.destroy()

with onto: 
    class Agent(Thing):
        pass
    class TiendaAgente(Agent):
        pass
    class CompradorAgente(Agent):
        pass
    class DineroEnTienda(Thing):
        pass
    class DineroComprador(Thing):
        pass
    class Productos(Thing):
        pass
    
    class obtener_dinero(FunctionalProperty, ObjectProperty):
        domain = [TiendaAgente]
        range = [DineroEnTienda]
        
    class dinero_value(FunctionalProperty, DataProperty):
        domain = [DineroEnTienda]
        range = [int]
    class AgenteFinanciero(Agent):
        pass
    
    class DineroFinanciero(Thing):
        pass   
    
    class has_inversion(FunctionalProperty, ObjectProperty):
        domain = [AgenteFinanciero]
        range = [DineroFinanciero]

    # Propiedad para representar la cantidad de dinero financiero (valor numérico)
    class inversion_value(FunctionalProperty, DataProperty):
        domain = [DineroFinanciero]
        range = [int]
    class dinero_disponible(FunctionalProperty, DataProperty):
        domain = [DineroComprador]
        range = [int]
        
    class realizar_compra(ObjectProperty):
        domain = [CompradorAgente]
        range = [Productos]
        
    class has_id(FunctionalProperty, DataProperty):
        domain = [Agent]
        range = [int]


In [14]:
onto.save(file="wealth_ontology.owl")

In [15]:
class Message():
    environment_buffer = []
    def __init__(self,sender=None,receiver=None,performative=None,content=None):
        self.sender = sender
        self.receiver = receiver
        self.performative = performative
        self.content = content
    def __str__(self):
        return f"\n\
        Sender: {self.sender}, \n\
        Receiver: {self.receiver}, \n\
        Performative: {self.performative}, \n\
        Content: {self.content}"
    def send(self):
        """
        The send function is used to send a message to the environment buffer.
        """
        Message.environment_buffer.append(self)


In [16]:
class TiendaAgente(ap.Agent):
    """Agente - Mariana"""
    def setup(self):
        self.recordedDinero = 0
        self.myself = onto.TiendaAgente()
        self.myself.has_id = self.id
        self.myself.has_dinero = onto.DineroEnTienda(dinero_value=0)
    
    def step(self):
        self.take_msg()
        self.recordedDinero = self.myself.has_dinero.dinero_value

    def take_msg(self):
        for msg in Message.environment_buffer:
            if msg.receiver == self.id:
                if msg.performative == "dardinero":
                    self.myself.has_dinero.dinero_value += msg.content["dardinero"]
                    Message.environment_buffer.remove(msg)
                elif msg.performative == "darpedido": 
                    self.dar_pedido(msg.content["cantidad"])

    
    def dar_pedido(self, num):
        #TODO: cambiar sender.id a repartidor
        msg = Message(sender=self.id, receiver=0, performative="dardinero",content={"item":1, "cantidad": num})
        msg.send()
    


In [17]:
class AgenteFinanciero(ap.Agent):
    """Agente que maneja dinero financiero, inversiones, etc."""
    def setup(self):

        self.inversionActual = 0
        
        self.myself = onto.AgenteFinanciero()
        self.myself.has_id = self.id
        
        self.myself.has_inversion = onto.DineroFinanciero(inversion_value=0)
    
    def step(self):
        """
        En cada paso, el agente revisa los mensajes y actualiza
        su conocimiento sobre el dinero financiero.
        """
        self.take_msg()
        self.inversionActual = self.myself.has_inversion.inversion_value
        
    def take_msg(self):
        
        for msg in Message.environment_buffer:
            if msg.receiver == self.id:
                
                if msg.performative == "invertir":
                    cantidad = msg.content.get("cantidad", 0)
                    self.myself.has_inversion.inversion_value += cantidad
                    print(f"AgenteFinanciero (ID {self.id}) recibe inversión de {cantidad}.")
                    
                    Message.environment_buffer.remove(msg)
  


In [26]:
class CompradorAgente(ap.Agent):
    """ Agente Comprador - Sofía """
    def setup(self):
        self.dinero = 200
        self.productos = []
        self.myself = onto.CompradorAgente()
        self.myself.has_id = self.id
        self.myself.dinero_disponible = onto.DineroComprador(dinero_value=200)

        
    def step(self):
        if self.myself.dinero_disponible.dinero_value > 0:
            total_compra = np.random.randint(1,80) 
            self.enviar_solicitud_compra(total_compra)
            
    def enviar_solicitud_compra(self, cantidad):
        msg = Message(sender=self.id, receiver=0, performative="comprar", content={"cantidad": cantidad})
        msg.send()
        self.myself.dinero_disponible.dinero_value -= cantidad
    
    def confirmacion_compra(self,msg):
        if msg.performative == "confimación":
            producto = msg.content.get("producto")
            if producto:
                self.productos.append(producto)
                print(f"Comprador {self.id} recibió: {producto}")
        
        
    

In [27]:
class VentasModel(ap.Model):
    def setup(self):
        self.agentsTienda = ap.AgentList(self, self.p.agents, TiendaAgente)
        #self.agentsComprador = ap.AgentList(self, self.p.agents, CompradorAgente)
    def step(self):
        self.agentsTienda.step()
        #self.agentsComprador.step()
    def update(self):
        self.record("Dinero obtenido", self.agentsTienda.recorded_Dinero)
    def end(self):
        return

In [28]:
parameters = {
    'agents': 100,
    'steps': 100
}

In [29]:
model = VentasModel(parameters)
results = model.run()

Completed: 100 steps
Run time: 0:00:00.031519
Simulation finished


In [30]:
results

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 2 keys
'variables': 
    'VentasModel': DataFrame with 1 variable and 101 rows
'reporters': DataFrame with 1 variable and 1 row
}